## 日本語版
※ ここではGoogle Colaraboratoryでの実行を想定しています。

In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [3]:
t_un.shape

torch.Size([11])

In [4]:
# t_u未知の単位の温度の値、と重み、バイアスからt_p(予測値)
def model(t_u, w, b):
    return w * t_u + b

In [5]:
# 予測値t_p, 摂氏温度t_cの誤差を求める
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [9]:
# requires_grad=True とすることでpramsのgrad属性を有効にしている
#params = torch.tensor([1.0, 0.0])
params = torch.tensor([1.0, 0.0], requires_grad=True)

In [10]:
# デフォルトでTrueになっている
params.grad is None

True

In [14]:
# dtypeも属性の1つ
params.dtype is None

False

In [18]:
# *をつけることでparamsの2つの値を読み込んでくれる
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

params.grad

tensor([9034.5938,  165.2000])

In [19]:
# grad属性を初期化する
if params.grad is not None:
    params.grad.zero_()

In [21]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # <1>
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():  # <2>
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params
# with構文の使い方(何かの処理を実行した時に自動的に(暗黙的に)実行処理をしてくれる)
# https://techacademy.jp/magazine/15823
# torch.no_grad()は勾配計算をしないようにする処理
# 上記ではその際に必ず、params.gradに学習率を乗じた値を減産してpramsに入れている
# backward処理をしたときに、grad属性を更新している
# このループはまず順伝播して損失を求め、backwrdで逆伝播より微分した変化量を求め
# それに学習率をかけて、再度順伝播して、逆伝播するというループ処理をしている
# https://qiita.com/tatsuya11bbs/items/86141fe3ca35bdae7338

In [22]:
# 上記を5000回繰り返す
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)